In [1]:
#%pip install ase==3.22.1
#%pip install numpy==1.24.4
#%pip install dscribe==2.5.0
#%pip install scipy

# Compare between each substances Euclid Distance and Cosine Similarity
# (SiO2 Quartz Sturcture and SiO2 cristobaliter Structure, GaAs wurtzite structure and GaAs Zinc Blend Structure) 
No SiO2 and GaAs comparison

In [ ]:
# Material stucture import from Material Project
# Input Material ptoject API key and structure key to generate position array
# Doesn't related to other cell in this ipynb

from pymatgen.ext.matproj import MPRester
from ase import Atoms
api_key = "4HwV8OwM6Tu8Q6wzS3m4HbT2X8vOtodP"
structure_key = "mp-8883"
def MatproImport(api_key,structure_key):
    # Set Materials Project API key
    
    # Get the structure for quartz SiO2 (mp-7000)
    with MPRester(api_key) as mpr:
        structure = mpr.get_structure_by_material_id(structure_key)

    #  Extract symbols and positions
    symbols = [str(site.specie) for site in structure.sites]
    positions = [site.coords for site in structure.sites]

    #Convert to ASE Atoms object (optional)
    ase_atoms = Atoms(symbols=symbols, positions=positions)

    # Print each element and its position
    '''
    print("Element positions (each in a row):")
    for symbol, pos in zip(symbols, positions):
        print(f"{symbol}: {pos}")
    print(positions)
    '''
    return positions
 
print(MatproImport(api_key,structure_key))

[array([-2.33346193e-06,  2.30570172e+00,  6.58600447e+00]), array([1.99679692, 1.15284913, 3.29294988]), array([-2.33346193e-06,  2.30570172e+00,  2.46330956e+00]), array([1.99679692, 1.15284913, 5.75636415])]


In [3]:
#Parameters setting
r_cut=5
n_max=2
l_max=0

In [ ]:
import numpy as np
from dscribe.descriptors import SOAP
from ase import Atoms
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

# Define the chemical species across all molecules
species = ["Si", "O", "Ga", "As"]

# Create SOAP descriptor with averaging mode
soap = SOAP(
    species=species,
    periodic=True,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    average="inner"  # Proper average mode
)

# Create molecular structures for SiO2 Quartz, Fused Silica, SiO2 Oxide Layer, GaAs Zinc Blend, and GaAs Wurtzite

# SiO2: Quartz (Hexagonal Structure)
si_o2_quartz = Atoms(
    symbols=["Si", "Si","Si", "O", "O", "O", "O", "O", "O"],
    positions=[[-1.2651302 ,  2.19127028,  1.84711592], [1.2651312 , 2.19127028, 3.69423739], [ 2.5302614e+00, -5.0000000e-09,  0.0000000e+00],[0.94678257, 3.46832126, 4.61779258],[2.53026451, 2.55409757, 2.77067665],[3.47704845, 0.9142193 , 0.92356073],[1.58347982, 0.9142193 , 4.61779258],[-2.11513671e-06,  1.82844299e+00,  2.77067665e+00],[-0.94678606,  3.46832126,  0.92356073]],
    cell=[4.92, 4.92, 5.43],  # Lattice parameter (approx)
    pbc=True
)
# SiO2: high (beta) Cristobaliter 
si_o2_cristobaliter  = Atoms(
    symbols=["Si", "Si","O", "O","O","O"],
    positions=[[5.54904951, 5.54904951, 5.54904951],[3.69936634, 3.69936634, 3.69936634],[2.77452476, 2.77452476, 4.62420793],[2.77452476, 4.62420793, 2.77452476],[4.62420793, 4.62420793, 4.62420793],[4.62420793, 2.77452476, 2.77452476]],
    cell=[5.276, 5.276, 5.276],  # Lattice parameter (approx)
    pbc=True
)

# GaAs: Zinc Blend structure (manual definition of atoms and positions)
ga_as_zinc_blend = Atoms(
    symbols=["Ga", "As"],
    positions=[[0., 0., 0.],[1.173751, 0.82996725, 2.03299675]],
    cell=[5.75, 5.75, 5.75],  # Lattice parameter (approx)
    pbc=True
)

# GaAs: Wurtzite structure (manual definition of atoms and positions)
ga_as_wurtzite = Atoms(
    symbols=["Ga","Ga", "As",  "As"],
    positions=[[-2.33346193e-06,  2.30570172e+00,  6.58600447e+00],[1.99679692, 1.15284913, 3.29294988],[-2.33346193e-06,  2.30570172e+00,  2.46330956e+00],[1.99679692, 1.15284913, 5.75636415]],
    cell=[3.99, 3.99, 6.59],  # Lattice parameter (approx)
    pbc=True
)

# Create SOAP descriptors (averaged over atoms)
vec_si_o2_quartz = soap.create(si_o2_quartz)
vec_si_o2_cristobaliter = soap.create(si_o2_cristobaliter)
vec_ga_as_zinc_blend = soap.create(ga_as_zinc_blend)
vec_ga_as_wurtzite = soap.create(ga_as_wurtzite)



# Create average vector for each substance

In [5]:
#local descriptor into a global descriptor is simply by taking the average over all sites
average_soap = SOAP(
    species=species,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    average="inner",
    sparse=False
)
#Sio2_quartz
average_si_o2_quartz = average_soap.create(si_o2_quartz)
print("Average SOAP si_o2_quartz",average_si_o2_quartz.shape)

#Sio2_cristobaliter
average_si_o2_cristobaliter = average_soap.create(si_o2_cristobaliter)
print("Average SOAP si_o2_cristobaliter",average_si_o2_cristobaliter.shape)

#GaAs_Wurtzite
average_ga_as_wurtzite = average_soap.create(ga_as_wurtzite)
print("Average SOAP ga_as_wurtzite",average_ga_as_wurtzite.shape)

#GaAs_Zinc_Blend
average_ga_as_zinc_blend = average_soap.create(ga_as_zinc_blend)
print("Average SOAP ga_as_zinc_blend",average_ga_as_zinc_blend.shape)

Average SOAP si_o2_quartz (36,)
Average SOAP si_o2_cristobaliter (36,)
Average SOAP ga_as_wurtzite (36,)
Average SOAP ga_as_zinc_blend (36,)


# Euclid Distance calculation

In [6]:
from scipy.spatial.distance import pdist, squareform
import numpy as np

molecules_si_o2 = np.vstack([average_si_o2_quartz, average_si_o2_cristobaliter])
distance_si_o2 = squareform(pdist(molecules_si_o2))
print("Distance matrix between SiO2: ")
print(distance_si_o2)
#Euclid Distance calculate
print("Euclid distance between SiO2: ",np.linalg.norm(distance_si_o2))
print("")

molecules_ga_as = np.vstack([average_ga_as_wurtzite, average_ga_as_zinc_blend])
distance_ga_as = squareform(pdist(molecules_ga_as))
print("Distance matrix between GaAs: ")
print(distance_ga_as)
#Euclid Distance calculate
print("Euclid distance between GaAs: ",np.linalg.norm(distance_ga_as))
print("")


Distance matrix between SiO2: 
[[0.         5.12437079]
 [5.12437079 0.        ]]
Euclid distance between SiO2:  7.24695466967479

Distance matrix between GaAs: 
[[ 0.        14.4982268]
 [14.4982268  0.       ]]
Euclid distance between GaAs:  20.50358896765264



# Cosine Similarity Calculation

The similarity can take values between -1 and +1. Smaller angles between vectors produce larger cosine values, indicating greater cosine similarity. 

cos(angle)= dot product between vectors /(L2 norm of vector1)* (L2 norm of vector2)

For example:
- When two vectors have the same orientation, the angle between them is 0, and the cosine similarity is 1.
- Perpendicular vectors have a 90-degree angle between them and a cosine similarity of 0.
- Opposite vectors have an angle of 180 degrees between them and a cosine similarity of -1.

Ref: https://www.learndatasci.com/glossary/cosine-similarity/

In [12]:
import math

def cos_sim(p, q):
    return np.dot(p, q) / (np.linalg.norm(p) * np.linalg.norm(q))

cos_sim_si_o2 =cos_sim(average_si_o2_quartz,average_si_o2_cristobaliter)
print("cos_sim_si_o2: ",cos_sim_si_o2)

cos_sim_ga_as =cos_sim(average_ga_as_wurtzite, average_ga_as_zinc_blend)
print("cos_sim_ga_as: ",cos_sim_ga_as)


cos_sim_si_o2:  0.99999100536915
cos_sim_ga_as:  0.9988064907593264
